In [2]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/merged_cleaned_train_data.csv'

try:
    df = pd.read_csv(file_path)

    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])

    print(f"Successfully loaded data from Drive. Shape: {df.shape}")
    print(df.head())

except FileNotFoundError:
    print(f"Error: File not found at {file_path}")

Mounted at /content/drive
Successfully loaded data from Drive. Shape: (93744, 11)
        country  brand_name month  months_postgx       volume  n_gxs  \
0  COUNTRY_B6AE  BRAND_1C1E   Jul            -24  272594.3921      0   
1  COUNTRY_B6AE  BRAND_1C1E   Aug            -23  351859.3103      0   
2  COUNTRY_B6AE  BRAND_1C1E   Sep            -22  447953.4813      0   
3  COUNTRY_B6AE  BRAND_1C1E   Oct            -21  411543.2924      0   
4  COUNTRY_B6AE  BRAND_1C1E   Nov            -20  774594.4542      0   

                                  ther_area  hospital_rate main_package  \
0  Muscoskeletal_Rheumatology_and_Osteology       0.002088         PILL   
1  Muscoskeletal_Rheumatology_and_Osteology       0.002088         PILL   
2  Muscoskeletal_Rheumatology_and_Osteology       0.002088         PILL   
3  Muscoskeletal_Rheumatology_and_Osteology       0.002088         PILL   
4  Muscoskeletal_Rheumatology_and_Osteology       0.002088         PILL   

   biological  small_molecule  
0 

Import Libraries

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.multioutput import MultiOutputRegressor
from google.colab import drive
from google.colab import files
import io

Best parameters (Taken by randomsearchCV)

In [4]:
FINAL_XGB_PARAMS = {
    'n_estimators': 2000, # was 400 earlier and changed it
    'learning_rate': 0.01, # thinking of changing this to a lower value
    'max_depth': 6,
    'min_child_weight': 5,
    'subsample': 0.6,
    'colsample_bytree': 1.0,
    'random_state': 42,
    'n_jobs': -1,
    'objective': 'reg:squarederror'
}

Path files and keywords

In [5]:
DRIVE_TRAIN_PATH = '/content/drive/MyDrive/merged_cleaned_train_data.csv'
TEST_FILE_NAME = '/content/drive/MyDrive/novartis_test_data_merged.csv'
KEYS_STATIC = ['country', 'brand_name']
static_cols = ['ther_area', 'main_package', 'biological', 'small_molecule', 'hospital_rate']

In [6]:
# helper functions
def log_transform(df): return np.log1p(df) # shrinks high volume data and stretches the small ones to make the data easier to learn from
def inverse_transform(arr): return np.expm1(arr) # bias correction when predictions were too low

# load data and prep
drive.mount('/content/drive')
df = pd.read_csv(DRIVE_TRAIN_PATH)

#drops any redundant columns
if 'Unnamed: 0' in df.columns: df = df.drop(columns=['Unnamed: 0'])
# replaces missing values with mean value to preserve row while maintaining the overall distribution of the variable
df['hospital_rate'] = df['hospital_rate'].fillna(df['hospital_rate'].mean())

# feature engineering
df_pre = df[(df['months_postgx'] >= -12) & (df['months_postgx'] <= -1)] # uses 12 months immediately preceding generic entry
avg_j = df_pre.groupby(KEYS_STATIC)['volume'].mean().reset_index(name='Avg_j') # used because PE is defined as the mean of the last 12 months before generic entry
df_static = df.drop_duplicates(subset=KEYS_STATIC)[KEYS_STATIC + static_cols]
X_base = pd.merge(df_static, avg_j, on=KEYS_STATIC, how='inner').set_index(KEYS_STATIC) #for scenario 1 prediction (uninformed features)

# X1, Y1 (M0..M23)
df_Y1 = df[(df['months_postgx'] >= 0) & (df['months_postgx'] <= 23)] #filters raw data to only include 24 months after generic entry
Y1 = df_Y1.pivot_table(index=KEYS_STATIC, columns='months_postgx', values='volume').fillna(np.nan) # target matrix for y1
X1 = pd.merge(X_base, Y1, left_index=True, right_index=True, how='inner').iloc[:, :-24]
Y1 = Y1.loc[X1.index] # created final target natrix

# X2, Y2 (M6..M23 with M0-M5 features) - Scenario 2 (Informed)

# feature engineering for scenario 2
df_M0_M5 = df[(df['months_postgx'] >= 0) & (df['months_postgx'] <= 5)].pivot_table(index=KEYS_STATIC, columns='months_postgx', values='volume').fillna(np.nan) # filters data to isolate actual volumes
M0_M5_feats = pd.concat([df_M0_M5[0].rename('Vol_M0'), df_M0_M5[5].rename('Vol_M5'), (df_M0_M5[0] - df_M0_M5[5]).rename('Erosion_M0_M5')], axis=1) # calculates initial erosion features months 0-6 post entry volume
X2 = pd.merge(X_base, M0_M5_feats, left_index=True, right_index=True, how='inner') # includes all input information necessary for the informed forecast from month 6 onwards

df_Y2 = df[(df['months_postgx'] >= 6) & (df['months_postgx'] <= 23)] # 18 month prediction
Y2 = df_Y2.pivot_table(index=KEYS_STATIC, columns='months_postgx', values='volume').fillna(np.nan)
Y2 = Y2.loc[X2.index] # created final target matrix

# encoding to prep data for XGBoost
X1['country_feat'] = X1.index.get_level_values('country')
X2['country_feat'] = X2.index.get_level_values('country')
cat_cols = X1.select_dtypes(include=['object']).columns.tolist()
preprocessor = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)], remainder='passthrough') # applied one hot encoding to use XGBoost efficiently
# fit and transform( maximizing data utilization)
X1_enc = preprocessor.fit_transform(X1).astype(float)
X2_enc = preprocessor.transform(X2).astype(float)

# Full Data Buckets (for training weights)
Y1_norm = Y1.div(X_base['Avg_j'], axis=0) # calculate normalized volume
buckets = np.where(Y1_norm.mean(axis=1) <= 0.25, 1, 2) # if bucket labelled 1 = high erosion
weights_full = np.where(buckets == 1, 2.0, 1.0) # this forces model to treat high erosion drugs error twice as costly

# Model Training
print("\n training on full dataset")
Y1_log = log_transform(Y1)
Y2_log = log_transform(Y2)

# Model 1 (S1)
m1 = MultiOutputRegressor(xgb.XGBRegressor(**FINAL_XGB_PARAMS))
m1.fit(X1_enc, Y1_log.values, sample_weight=weights_full)

# Model 2 (S2)
m2 = MultiOutputRegressor(xgb.XGBRegressor(**FINAL_XGB_PARAMS))
m2.fit(X2_enc, Y2_log.values, sample_weight=weights_full)
print("Final Tuned Models Trained Successfully.")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- FINAL MODEL TRAINING ON FULL DATASET (2000 Estimators) ---
Final Tuned Models Trained Successfully.

--- Generating Submission CSV ---


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submission Complete! Final CSV 'team51_test_submission_tuned.csv' created with 7488 rows.


In [ ]:
# Submission CSV
try:
    df_test = pd.read_csv(TEST_FILE_NAME)
except FileNotFoundError:
    print(f"Error: '{TEST_FILE_NAME}' not found.")
    raise

# initial cleaning (Test)
df_test['n_gxs'] = df_test['n_gxs'].fillna(0.0)
df_test['hospital_rate'] = df_test['hospital_rate'].fillna(df_test['hospital_rate'].mean())

# test Feature Engineering (perfectly match train FE)
df_pre_t = df_test[(df_test['months_postgx'] >= -12) & (df_test['months_postgx'] <= -1)]
avg_j_t = df_pre_t.groupby(KEYS_STATIC)['volume'].mean().reset_index(name='Avg_j')
df_static_t = df_test.drop_duplicates(subset=KEYS_STATIC)[KEYS_STATIC + static_cols]
X_base_t = pd.merge(df_static_t, avg_j_t, on=KEYS_STATIC, how='inner').set_index(KEYS_STATIC)
df_M0_M5_t = df_test[(df_test['months_postgx'] >= 0) & (df_test['months_postgx'] <= 5)].pivot_table(index=KEYS_STATIC, columns='months_postgx', values='volume').fillna(np.nan)
M0_M5_feats_t = pd.concat([df_M0_M5_t[0].rename('Vol_M0'), df_M0_M5_t[5].rename('Vol_M5'), (df_M0_M5_t[0] - df_M0_M5_t[5]).rename('Erosion_M0_M5')], axis=1)

# split Test drugs
s2_drugs = M0_M5_feats_t.dropna().index
s1_drugs = X_base_t.index.difference(s2_drugs)

# prepare final prediction matrices
X_test_S1 = X_base_t.loc[s1_drugs].copy()
X_test_S1['country_feat'] = X_test_S1.index.get_level_values('country')
X_test_S2 = pd.merge(X_base_t, M0_M5_feats_t, left_index=True, right_index=True, how='inner').loc[s2_drugs]
X_test_S2['country_feat'] = X_test_S2.index.get_level_values('country')

# encoding
X_test_S1_enc = preprocessor.transform(X_test_S1).astype(float)
X_test_S2_enc = preprocessor.transform(X_test_S2).astype(float)

# predict
p_s1_log = m1.predict(X_test_S1_enc)
p_s2_log = m2.predict(X_test_S2_enc)

p_s1_final = np.maximum(0, inverse_transform(p_s1_log))
p_s2_final = np.maximum(0, inverse_transform(p_s2_log))

# Combine and Save
df_p1 = pd.DataFrame(p_s1_final, index=X_test_S1.index, columns=[f'Volume_M{i}' for i in range(24)])
df_p2 = pd.DataFrame(p_s2_final, index=X_test_S2.index, columns=[f'Volume_M{i}' for i in range(6, 24)])

p1_long = df_p1.stack().reset_index()
p1_long.columns = ['country', 'brand_name', 'm_col', 'volume']
p1_long['months_postgx'] = p1_long['m_col'].str.replace('Volume_M', '').astype(int)

p2_long = df_p2.stack().reset_index()
p2_long.columns = ['country', 'brand_name', 'm_col', 'volume']
p2_long['months_postgx'] = p2_long['m_col'].str.replace('Volume_M', '').astype(int)

final_submission = pd.concat([p1_long[['country', 'brand_name', 'months_postgx', 'volume']], p2_long[['country', 'brand_name', 'months_postgx', 'volume']]])

filename = 'team51_test_submission_tuned.csv'
final_submission.to_csv(filename, index=False)
files.download(filename)
print(f"Submission Complete! Final CSV '{filename}' created with {len(final_submission)} rows.")

Metrics calculation (From Novartis)

In [29]:
import pandas as pd
import numpy as np

# successfully defined:
# X1_val, Y1_val, X2_val, Y2_val (Validation feature/target sets)
# m1, m2 (Trained XGBoost models)
# df_aux_val (Auxiliary data with bucket/avg_vol)
# log_transform, inverse_transform (Helper functions)

def _compute_pe_phase1a(group: pd.DataFrame) -> float:

    avg_vol = group["avg_vol"].iloc[0];
    if avg_vol == 0 or np.isnan(avg_vol): return np.nan
    def sum_abs_diff(month_start: int, month_end: int) -> float:
        subset = group[(group["months_postgx"] >= month_start) & (group["months_postgx"] <= month_end)];
        return (subset["volume_actual"] - subset["volume_predict"]).abs().sum()
    def abs_sum_diff(month_start: int, month_end: int) -> float:
        subset = group[(group["months_postgx"] >= month_start) & (group["months_postgx"] <= month_end)];
        return abs(subset["volume_actual"].sum() - subset["volume_predict"].sum())
    term1 = 0.2 * sum_abs_diff(0, 23) / (24 * avg_vol);
    term2 = 0.5 * abs_sum_diff(0, 5) / (6 * avg_vol);
    term3 = 0.2 * abs_sum_diff(6, 11) / (6 * avg_vol);
    term4 = 0.1 * abs_sum_diff(12, 23) / (12 * avg_vol);
    return term1 + term2 + term3 + term4

def compute_metric1(df_actual: pd.DataFrame, df_pred: pd.DataFrame, df_aux: pd.DataFrame) -> float:

    merged = df_actual.merge(df_pred, on=["country", "brand_name", "months_postgx"], how="inner", suffixes=("_actual", "_predict")).merge(df_aux, on=["country", "brand_name"], how="left");
    merged["start_month"] = merged.groupby(["country", "brand_name"])["months_postgx"].transform("min");
    merged = merged[merged["start_month"] == 0].copy();
    pe_results = (merged.groupby(["country", "brand_name", "bucket"]).apply(_compute_pe_phase1a).reset_index(name="PE"));
    bucket1 = pe_results[pe_results["bucket"] == 1]; bucket2 = pe_results[pe_results["bucket"] == 2];
    n1 = len(bucket1); n2 = len(bucket2); score = 0;
    if n1 > 0: score += (2 / n1) * bucket1["PE"].sum();
    if n2 > 0: score += (1 / n2) * bucket2["PE"].sum();
    return round(score, 4)

def _compute_pe_phase1b(group: pd.DataFrame) -> float:

    avg_vol = group["avg_vol"].iloc[0];
    if avg_vol == 0 or np.isnan(avg_vol): return np.nan
    def sum_abs_diff(month_start: int, month_end: int) -> float:
        subset = group[(group["months_postgx"] >= month_start) & (group["months_postgx"] <= month_end)];
        return (subset["volume_actual"] - subset["volume_predict"]).abs().sum()
    def abs_sum_diff(month_start: int, month_end: int) -> float:
        subset = group[(group["months_postgx"] >= month_start) & (group["months_postgx"] <= month_end)];
        return abs(subset["volume_actual"].sum() - subset["volume_predict"].sum())
    term1 = 0.2 * sum_abs_diff(6, 23) / (18 * avg_vol);
    term2 = 0.5 * abs_sum_diff(6, 11) / (6 * avg_vol);
    term3 = 0.3 * abs_sum_diff(12, 23) / (12 * avg_vol);
    return term1 + term2 + term3

def compute_metric2(df_actual: pd.DataFrame, df_pred: pd.DataFrame, df_aux: pd.DataFrame) -> float:

    merged_data = df_actual.merge(df_pred, on=["country", "brand_name", "months_postgx"], how="inner", suffixes=("_actual", "_predict")).merge(df_aux, on=["country", "brand_name"], how="left");
    merged_data["start_month"] = merged_data.groupby(["country", "brand_name"])["months_postgx"].transform("min");
    merged_data = merged_data[merged_data["start_month"] == 6].copy();
    pe_results = (merged_data.groupby(["country", "brand_name", "bucket"]).apply(_compute_pe_phase1b).reset_index(name="PE"));
    bucket1 = pe_results[pe_results["bucket"] == 1]; bucket2 = pe_results[pe_results["bucket"] == 2];
    n1 = len(bucket1); n2 = len(bucket2); score = 0;
    if n1 > 0: score += (2 / n1) * bucket1["PE"].sum();
    if n2 > 0: score += (1 / n2) * bucket2["PE"].sum();
    return round(score, 4)

Generate Predictions and Final Scores

Calculation

In [35]:
from sklearn.model_selection import train_test_split

# use integer indices to split (keep original MultiIndex for aux) ---
#TODO ; FIX THIS TO TRAIN TEST SPLIT
val_frac = 0.2
val_idx = np.random.choice(X1.shape[0], size=int(val_frac*X1.shape[0]), replace=False)
train_idx = np.setdiff1d(np.arange(X1.shape[0]), val_idx)

# Split original X1, X2 (not encoded arrays) for indexing
X1_val_idx = X1.iloc[val_idx]
Y1_val_idx = Y1.iloc[val_idx]
X2_val_idx = X2.iloc[val_idx]
Y2_val_idx = Y2.iloc[val_idx]

# Also prepare aux data
df_aux_val = pd.DataFrame({
    'country': X1_val_idx.index.get_level_values('country'),
    'brand_name': X1_val_idx.index.get_level_values('brand_name'),
    'bucket': buckets[val_idx],
    'avg_vol': X_base.loc[X1_val_idx.index, 'Avg_j'].values
})

# predict using the encoded arrays for validation ---
preds_real_s1 = np.maximum(0, inverse_transform(m1.predict(X1_enc[val_idx])))
preds_real_s2 = np.maximum(0, inverse_transform(m2.predict(X2_enc[val_idx])))

# format long DataFrames correctly ---
# Scenario 1
df_actual_S1 = Y1_val_idx.stack().reset_index()
df_actual_S1 = df_actual_S1.rename(columns={0: 'volume_actual', 'months_postgx': 'm_col'})
df_actual_S1['months_postgx'] = df_actual_S1['m_col']

df_pred_S1 = pd.DataFrame(preds_real_s1, index=Y1_val_idx.index, columns=Y1_val_idx.columns).stack().reset_index()
df_pred_S1 = df_pred_S1.rename(columns={0: 'volume_predict', 'months_postgx': 'm_col'})
df_pred_S1['months_postgx'] = df_pred_S1['m_col']

# Scenario 2
df_actual_S2 = Y2_val_idx.stack().reset_index()
df_actual_S2 = df_actual_S2.rename(columns={0: 'volume_actual', 'months_postgx': 'm_col'})
df_actual_S2['months_postgx'] = df_actual_S2['m_col']

df_pred_S2 = pd.DataFrame(preds_real_s2, index=Y2_val_idx.index, columns=Y2_val_idx.columns).stack().reset_index()
df_pred_S2 = df_pred_S2.rename(columns={0: 'volume_predict', 'months_postgx': 'm_col'})
df_pred_S2['months_postgx'] = df_pred_S2['m_col']

# compute PE Scores ---
m1_score_final = compute_metric1(
    df_actual_S1[['country', 'brand_name', 'months_postgx', 'volume_actual']],
    df_pred_S1[['country', 'brand_name', 'months_postgx', 'volume_predict']],
    df_aux_val
)

m2_score_final = compute_metric2(
    df_actual_S2[['country', 'brand_name', 'months_postgx', 'volume_actual']],
    df_pred_S2[['country', 'brand_name', 'months_postgx', 'volume_predict']],
    df_aux_val
)

print(f"Final PE Score (Scenario 1, S1): {m1_score_final}")
print(f"Final PE Score (Scenario 2, S2): {m2_score_final}")


Final PE Score (Scenario 1, S1): 0.1104
Final PE Score (Scenario 2, S2): 0.122


/tmp/ipython-input-4014683630.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pe_results = (merged.groupby(["country", "brand_name", "bucket"]).apply(_compute_pe_phase1a).reset_index(name="PE"));
/tmp/ipython-input-4014683630.py:58: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pe_results = (merged_data.groupby(["country", "brand_name", "bucket"]).apply(_compute_pe_phase1b).reset_index(name="PE"));


In [36]:
# # TODO FIX THE TRAIN TEST SPLIT


# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split

# # --- ASSUMES: df, X_base, Y1, and buckets are already defined ---

# # --- 1. Define df_aux_full (Auxiliary Data for Buckets and Avg_Vol) ---

# # Re-calculate Avg_j (Mean Volume M-12 to M-1)
# # This uses the original X_base (which contains Avg_j)
# df_avg = X_base[['Avg_j']].reset_index()

# # Recalculate buckets and create the full auxiliary DataFrame
# # Ensure you are using the 'buckets' array that was defined in your previous step
# df_aux_full = X_base[['Avg_j']].copy().rename(columns={'Avg_j': 'avg_vol'})
# df_aux_full['bucket'] = np.where(Y1.div(X_base['Avg_j'], axis=0).mean(axis=1) <= 0.25, 1, 2)
# df_aux_full = df_aux_full.reset_index().set_index(['country', 'brand_name'])


# # --- 2. Perform CORRECT STRATIFIED Validation Split ---

# # Get the list of ALL unique drug indices
# original_indices = df_aux_full.index.values

# # Perform STRATIFIED Split on the Indices
# train_indices_str, val_indices_str = train_test_split(
#     original_indices,
#     test_size=0.2,
#     random_state=42,
#     # CRITICAL: Stratify using the calculated bucket values
#     stratify=df_aux_full['bucket'].values
# )

# # Map string indices to integer positions (needed for slicing encoded NumPy arrays)
# idx_map = {idx: i for i, idx in enumerate(X1.index.values)}

# train_indices_int = [idx_map[idx] for idx in train_indices_str]
# val_indices_int = [idx_map[idx] for idx in val_indices_str]


# # --- 3. Final Data Slicing ---

# # X matrices (Encoded features - use integer index)
# X1_train = X1_enc[train_indices_int]
# X1_val = X1_enc[val_indices_int]

# # Y matrices (Targets - use the string index for the DataFrame slice)
# Y1_train = Y1.loc[train_indices_str]
# Y1_val = Y1.loc[val_indices_str]

# # Auxiliary Data for Metric Calculation
# df_aux_val = df_aux_full.loc[val_indices_str].reset_index()
# weights_tr = np.where(df_aux_full.loc[train_indices_str]['bucket'].values == 1, 2.0, 1.0)


# print("✅ df_aux_full defined and Stratified Validation Split is complete.")

✅ df_aux_full defined and Stratified Validation Split is complete.


0.2 has been an improvement ( from 0.6 using 400 num_estimates)

In [7]:
# # to check encoding
# # 1. Print UNENCODED Features (Pandas DataFrame)
# # Shows the original categories and numeric values.
# print("\n[A] UNENCODED FEATURES (X1 - First 5 Rows):")
# print(X1.head())
# print("-" * 30)
# print(f"Original X1 Shape: {X1.shape}")

# # 2. Print ENCODED Features (NumPy Array)
# # This shows the final numerical result fed into the XGBoost model.
# print("\n[B] ENCODED FEATURES (X1_enc - First 5 Rows of NumPy Array):")
# print(X1_enc[:5, :10]) # Print first 5 rows and first 10 columns
# print(f"Final X1_enc Shape: {X1_enc.shape}")
# print("-" * 30)

# # Optional: Print total number of features created (columns in the final array)
# print(f"Total Features Created After Encoding: {X1_enc.shape[1]}")



--- 🔬 ENCODING DIAGNOSTIC ---

[A] UNENCODED FEATURES (X1 - First 5 Rows):
                                                        ther_area  \
country      brand_name                                             
COUNTRY_B6AE BRAND_1C1E  Muscoskeletal_Rheumatology_and_Osteology   
             BRAND_E7C2                           Anti-infectives   
             BRAND_9065                  Cardiovascular_Metabolic   
             BRAND_022B       Respiratory_and_Immuno-inflammatory   
             BRAND_D8CE       Endocrinology_and_Metabolic_Disease   

                        main_package  biological  small_molecule  \
country      brand_name                                            
COUNTRY_B6AE BRAND_1C1E         PILL       False            True   
             BRAND_E7C2         PILL       False            True   
             BRAND_9065         PILL       False            True   
             BRAND_022B       Others       False            True   
             BRAND_D8CE        